# Chrome Built-in AI Demo in JupyterLab and JupyterLite Python Notebook

This notebook demonstrates the Python wrapper for Chrome's built-in AI Prompt API.

https://developer.chrome.com/docs/ai/prompt-api

**Requirements:**
- Chrome browser with Prompt API enabled
- Running in JupyterLab, Jupyter Notebook, or JupyterLite (statically served, all execution locally in browser)

## For developers: Using the Prompt API
* Open Chrome and go to `chrome://flags`.
* Enable the `#prompt-api-for-gemini-nano` flag.
* Enable the `#optimization-guide-on-device-model` flag. If you see a "BypassPerfRequirement" option, select it.
* Restart Chrome.

In [1]:
%pip install -q anywidget ipywidgets wiki3_ai

Note: you may need to restart the kernel to use updated packages.


In [2]:
%gui asyncio

In [3]:
# Import the library
from wiki3_ai import LanguageModel, Availability, LanguageModelWidget

In [4]:
lm_widget = LanguageModelWidget()
lm_widget

In [5]:
lm = await LanguageModel.create(lm_widget)
lm

## Check Availability

First, let's check if the Prompt API is available in your browser.

In [6]:
# Check if the API is available
await lm.availability()

<Availability.AVAILABLE: 'available'>

## Simple Prompt

Create a session and send a simple prompt.

In [7]:
# Send a prompt
result = await lm.prompt("Write a haiku about Python programming.")
print(result)

Clean syntax flows free,
Logic blooms, a coded dream,
Scripts come alive now. 






## System Prompt

Use a system prompt to set the context for the conversation.

In [8]:
# Create a session with a system prompt
assistant_session = await LanguageModel.create(lm_widget,
    {
        "initialPrompts": [
            {
                "role": "system",
                "content": "You are a helpful Python programming assistant who gives concise answers.",
            }
        ]
    }
)

# Ask a question
result = await assistant_session.prompt("How do I read a CSV file in Python?")
print(result)

```python
import csv

with open('your_file.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        print(row)
```

**Explanation:**

1.  **`import csv`**: Imports the `csv` module.
2.  **`with open('your_file.csv', 'r') as file:`**: Opens the CSV file in read mode (`'r'`).  The `with` statement ensures the file is automatically closed. Replace `'your_file.csv'` with the actual file name.
3.  **`reader = csv.reader(file)`**: Creates a CSV reader object.
4.  **`for row in reader:`**: Iterates through each row in the CSV file.
5.  **`print(row)`**: Prints each row (as a list of strings).



For more complex CSV files (e.g., with headers, different delimiters), explore `csv.DictReader` and the `csv` module's documentation.


## Streaming Response

Stream the response as it's generated.

In [9]:
print("Response: ", end="")
async for chunk in assistant_session.prompt_streaming("Tell me a short joke about computers."):
    print(chunk, end="", flush=True)
print()

Response: 


## Multi-turn Conversation

Have a conversation with context maintained across prompts.

In [10]:
# First message
result1 = await assistant_session.prompt("What is a list comprehension?")
print("Assistant:", result1)
print()

# Follow-up message (the assistant remembers the context)
result2 = await assistant_session.prompt("Can you show me an example?")
print("Assistant:", result2)

Assistant: A list comprehension is a concise way to create lists in Python. It's a more compact alternative to using a `for` loop and `append()`.  It allows you to generate a new list by applying an expression to each item in an existing iterable (like a list, tuple, or range).

**Example:**

```python
# Traditional loop
squares = []
for i in range(10):
  squares.append(i**2)

# List comprehension
squares = [i**2 for i in range(10)]

print(squares)  # Output: [0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
```

In short, it's a shorthand for creating lists based on existing iterables.

Assistant: ```python
# Example: Extract even numbers from a list

numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# List comprehension to get even numbers
even_numbers = [num for num in numbers if num % 2 == 0]

print(even_numbers)  # Output: [2, 4, 6, 8, 10]

# Example: Convert strings to uppercase

words = ["hello", "world", "python"]

uppercase_words = [word.upper() for word in words]

print(uppercase_words) # Output

## Check Token Usage

Monitor how many tokens you've used.

In [11]:
print(f"Current usage: {assistant_session.input_usage}/{assistant_session.input_quota} tokens")

# Measure how many tokens a prompt would use
usage = await assistant_session.measure_input_usage("What is machine learning?")
print(f"\nThis prompt would use approximately {usage} tokens")

Current usage: 691/9216 tokens

This prompt would use approximately 13 tokens


## Get Model Parameters

Let's see what parameters the model supports.

In [12]:
params = await lm.params()
params

LanguageModelParams(default_top_k=3, max_top_k=128, default_temperature=1, max_temperature=2)

In [13]:
if params:
    print(f"Default temperature: {params.default_temperature}")
    print(f"Max temperature: {params.max_temperature}")
    print(f"Default top-K: {params.default_top_k}")
    print(f"Max top-K: {params.max_top_k}")
else:
    print("Model parameters not available")

Default temperature: 1
Max temperature: 2
Default top-K: 3
Max top-K: 128


## Structured Output

Use JSON schema to get structured responses.

In [14]:
import json

# Define a JSON schema
schema = {
    "type": "object",
    "required": ["sentiment", "score"],
    "properties": {
        "sentiment": {"type": "string", "enum": ["positive", "negative", "neutral"]},
        "score": {"type": "number", "minimum": 0, "maximum": 1},
    },
}

# Get structured response
result = await lm.prompt(
    "Analyze this review: The product exceeded all my expectations! Absolutely amazing!",
    {"responseConstraint": schema},
)

# Parse the JSON response
data = json.loads(result)
print(f"Sentiment: {data['sentiment']}")
print(f"Score: {data['score']}")

Sentiment: positive
Score: 0.95


## Session Cloning

Clone a session to create different conversation branches.

In [15]:
# Start a story
story_session = await LanguageModel.create(lm_widget,
    {"initialPrompts": [{"role": "system", "content": "You are a creative storyteller."}]}
)

await story_session.prompt("Once upon a time, there was a dragon.")

# Create two different story branches
branch1 = await story_session.clone()
branch2 = await story_session.clone()

result1 = await branch1.prompt("The dragon was friendly and helpful.")
print("Branch 1:", result1)
print()

result2 = await branch2.prompt("The dragon was fierce and terrifying.")
print("Branch 2:", result2)

Branch 1: Once upon a time, there was a dragon named Zephyr, but he wasn't your typical, fire-breathing, hoard-guarding kind. Zephyr was a collector of lost melodies. Instead of gold, he amassed fragments of songs – a whisper of a lullaby caught on the wind, a single, shimmering chord from a forgotten flute, the echo of a joyous tavern song fading into twilight. 

Zephyr lived not in a volcanic mountain, but nestled amongst the whispering willows of the Sunken Glade, a place where moonlight always seemed to linger. His scales weren't the usual crimson or emerald, but a soft, iridescent lavender, shifting with the colors of the dawn. And his breath? It wasn’t fire, but a delicate, shimmering mist that could weave forgotten notes back into existence.

He wasn’t a fearsome guardian, not at all. Zephyr was incredibly friendly and helpful. He’d happily assist lost travelers, guiding them through tangled forests with a gentle hum that resonated with the path. He'd mend broken instruments wit

## Cleanup

Destroy sessions when you're done to free up resources.

In [16]:
await lm.destroy()
await assistant_session.destroy()
await story_session.destroy()
await branch1.destroy()
await branch2.destroy()

print("✅ All sessions destroyed")

✅ All sessions destroyed
